# Image pre-reduction

This page will take us through the full pre-reduction of the science images.

You could copy or write the code shown below in a script, or alternatively directly download this page as a {download}`jupyter notebook <./pre-red.ipynb>` file.

Before running the code, make sure you've created and adjusted the module `dataredconfig.py`, as explained [here](./data.md).


In [ ]:
import dataredconfig

from pathlib import Path
import numpy as np
import astropy
from astropy import units as u
%matplotlib ipympl
import matplotlib
from matplotlib import pyplot as plt
import ccdproc
import photutils

print("numpy", np.__version__, ", astropy", astropy.__version__, ", matplotlib", matplotlib.__version__, ", ccdproc", ccdproc.__version__, ", photutils", photutils.__version__)
# Developed with (April 2024): numpy 1.26.4 , astropy 6.0.1 , matplotlib 3.8.3 , ccdproc 2.4.1 , photutils 1.11.0

In [ ]:
# We'll ignore some astropy warnings that get raised as our FITS headers (from NINA) are not 100% standards compliant.
import warnings
warnings.simplefilter('ignore', category=astropy.wcs.FITSFixedWarning)

## Creating a master-bias

In [ ]:
bias_files = ccdproc.ImageFileCollection(dataredconfig.data_dir / "BIAS", keywords=dataredconfig.ifc_header_keywords)
bias_files.summary

In [ ]:
# We check the level and noise in these files:
for ccd, filename in bias_files.ccds(ccd_kwargs={'unit': 'adu'}, return_fname=True):
    print(f"{filename}: standard deviation {ccd.data.std():.2f} ADU, median level {np.median(ccd.data):.2f} ADU")

In [ ]:
combiner = ccdproc.Combiner(bias_files.ccds(ccd_kwargs={"unit":"adu"}))
masterbias = combiner.average_combine()

masterbias.meta['combined'] = True
masterbias.data = masterbias.data.astype('float32') # Converts to float32 to save space
masterbias.write(dataredconfig.work_dir / 'masterbias.fits', overwrite=True)

print(f"masterbias: standard deviation {masterbias.data.std():.2f} ADU, median level {np.median(masterbias.data):.2f} ADU")

In [ ]:
# Let's check for some structure, by median-smoothing this a bit:
smooth_masterbias = photutils.background.Background2D(masterbias.data, (100, 100), filter_size=(3, 3), bkg_estimator=photutils.background.MedianBackground()).background

plt.figure()
plt.imshow(smooth_masterbias, origin='lower', cmap='Greys_r', interpolation='nearest')
plt.tight_layout()
plt.show()

## Creating a master-dark

In [ ]:
dark_files = ccdproc.ImageFileCollection(dataredconfig.data_dir / "DARK", keywords=dataredconfig.ifc_header_keywords)
dark_files.summary


In [ ]:
# For this simple demo we want them to all have the same exptime!
exptimes = list(set(dark_files.summary["exptime"]))
assert(len(exptimes) == 1)
dark_exptime = exptimes[0]
print("Exposure times of darks: ", dark_exptime)

# As done for the bias frames, let's check the level and noise in these darks:
# Question to students, can you guess why the std is so much higher than for bias frames, while the level is almost the same?

for ccd, filename in dark_files.ccds(ccd_kwargs={'unit': 'adu'}, return_fname=True):
    print(f"{filename}: standard deviation {ccd.data.std():.2f} ADU, median level {np.median(ccd.data):.2f} ADU")

In [ ]:
# Create the masterdark

dest_dir = dataredconfig.work_dir / "DARK_BIASSUB"
dest_dir.mkdir(exist_ok=True)

# First loop over the dark frames to subtract the bias: 
for ccd, filename in dark_files.ccds(ccd_kwargs={'unit': 'adu'}, return_fname=True):
    print(f"Processing {filename}...")
    ccd = ccdproc.subtract_bias(ccd, masterbias)
    # Write to disk:
    ccd.data = ccd.data.astype('float32')
    ccd.write(dest_dir / filename, overwrite=True)

# Note: depending on the available memory,
# it might be possible to do this without writing intermediary files,
# following something like:
#dark_ccds = [ccdproc.subtract_bias(ccd, masterbias) for ccd in dark_files.ccds(ccd_kwargs={'unit': 'adu'})]

# Then we combine the files:
files_to_combine = ccdproc.ImageFileCollection(dest_dir).files_filtered(include_path=True)
masterdark = ccdproc.combine(files_to_combine,
                                 method='average',
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=astropy.stats.mad_std,
                                 mem_limit=1e9
                                )

masterdark.meta['combined'] = True
masterdark.data = masterdark.data.astype('float32') # Converts to float32 to save space
masterdark.write(dataredconfig.work_dir / 'masterdark.fits', overwrite=True)


In [ ]:
# Let's check for some structure, by filtering-out hot pixels:

smooth_masterdark = photutils.background.Background2D(masterdark.data, (200, 200), filter_size=(1, 1), bkg_estimator=photutils.background.MedianBackground()).background

plt.figure()
plt.imshow(smooth_masterdark, origin='lower', cmap='Greys_r', interpolation='nearest')
plt.tight_layout()
plt.show()


## Creating the master-flats

In [ ]:
flat_files = ccdproc.ImageFileCollection(dataredconfig.data_dir / "FLAT", keywords=dataredconfig.ifc_header_keywords)
flat_files.summary


In [ ]:
# Scaling function for the flats: they get "normalized" (i.e., divided) by their median:
def inv_median(a):
    return 1 / np.median(a)

In [ ]:

# We'll simply loop over these filters:
flat_filters_to_run_on = ["g", "r", "i"]

for selected_filter in flat_filters_to_run_on:

    selected_flat_files = flat_files.filter(filter=selected_filter)
    
    dest_dir = dataredconfig.work_dir / f"FLAT_{selected_filter}_BIASDARKSUB"
    dest_dir.mkdir(exist_ok=True)

    # First subtract bias and dark from every flat:
    for ccd, filename in selected_flat_files.ccds(ccd_kwargs={'unit': 'adu'}, return_fname=True):
        print(f"Processing {filename}...")
        ccd = ccdproc.subtract_bias(ccd, masterbias)
        ccd = ccdproc.subtract_dark(ccd, masterdark, exposure_time='exptime', exposure_unit=u.second, scale=True)
    
        # Write to disk:
        ccd.data = ccd.data.astype('float32') # Converts to float32 to save space
        ccd.write(dest_dir / filename, overwrite=True)


    # And now combine to a masterflat:
    files_to_combine = ccdproc.ImageFileCollection(dest_dir).files_filtered(include_path=True)
    masterflat = ccdproc.combine(files_to_combine,
                                 method='average', scale=inv_median,
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=astropy.stats.mad_std,
                                 mem_limit=1e9
                                )

    masterflat.meta['combined'] = True
    masterflat.data = masterflat.data.astype('float32') # Converts to float32 to save space
    masterflat.write(dataredconfig.work_dir / f"masterflat_{selected_filter}.fits", overwrite=True)

In [ ]:
# Checking the masterflats

masterflat = ccdproc.CCDData.read(dataredconfig.work_dir / "masterflat_g.fits")
plt.figure()
plt.imshow(masterflat.data, vmin=0.9, vmax=1.05, origin='lower', cmap='Greys_r', interpolation='nearest')
plt.tight_layout()
plt.show()


## Processing the science images

In [ ]:
science_files = ccdproc.ImageFileCollection(dataredconfig.data_dir / "LIGHT", keywords=dataredconfig.ifc_header_keywords)
science_files.summary

In [ ]:

# Defining the directory where pre-reduced images should be written:
dest_dir = dataredconfig.work_dir / "LIGHT_PRERED"
dest_dir.mkdir(exist_ok=True)

# Reading in from disk the master calibration frames:
masterbias = ccdproc.CCDData.read(dataredconfig.work_dir / "masterbias.fits")
masterdark = ccdproc.CCDData.read(dataredconfig.work_dir / "masterdark.fits")
# For the flats, we build a dictionnary that holds the masterflat for each filter name:
available_flat_filters = ["r", "g", "i"]
masterflats = {filtername: ccdproc.CCDData.read(dataredconfig.work_dir / f"masterflat_{filtername}.fits") for filtername in available_flat_filters}

# We loop over all files, selecting the right masterflat on the fly
for ccd, filename in science_files.ccds(ccd_kwargs={'unit': 'adu'}, return_fname=True):
    print(f"Processing {filename}...")
    ccd = ccdproc.subtract_bias(ccd, masterbias)
    ccd = ccdproc.subtract_dark(ccd, masterdark, exposure_time='exptime', exposure_unit=u.second, scale=True)
    
    masterflat_to_use = masterflats[ccd.header['filter']]
    ccd = ccdproc.flat_correct(ccd, masterflat_to_use)
    
    # Write to disk:
    ccd.data = ccd.data.astype('float32') # Converts to float32 to save space
    ccd.write(dest_dir / filename, overwrite=True)